In [17]:
import pandas as pd
import polars as pl

In [20]:
pd.read_csv("/home/egor/Projects/Pycharm/kaggle-detect_sleep_states/data/raw/train_events.csv").dropna().groupby("series_id")["event"].count().describe()

count    269.000000
mean      35.631970
std       16.583188
min        2.000000
25%       24.000000
50%       38.000000
75%       46.000000
max       70.000000
Name: event, dtype: float64

In [39]:
pd.read_csv("/home/egor/Projects/Pycharm/kaggle-detect_sleep_states/data/raw/train_events.csv").dropna().groupby("series_id")["step"].max().describe()

count       269.000000
mean     364601.174721
std      144422.091716
min        9132.000000
25%      291600.000000
50%      377544.000000
75%      427764.000000
max      739392.000000
Name: step, dtype: float64

In [33]:
df = pl.scan_parquet("/home/egor/Projects/Pycharm/kaggle-detect_sleep_states/data/raw/train_series.parquet", low_memory=True)
df

<LazyFrame [5 cols, {"series_id": Utf8 … "enmo": Float32}] at 0x7F9571990940>

In [35]:
df_test = df.filter(pl.col("series_id").is_in(pl.col("series_id").unique().gather([0,1,2])))
df_test

<LazyFrame [5 cols, {"series_id": Utf8 … "enmo": Float32}] at 0x7F95AA4702E0>

In [37]:
df_test.collect().write_parquet("/home/egor/Projects/Pycharm/kaggle-detect_sleep_states/data/raw/test_series.parquet")

In [15]:
df.group_by("series_id").agg(pl.col("step").max()).select(pl.col("step").min()).collect()

step
u32
37079


In [16]:
df.group_by("series_id").agg(
    pl.col("step").diff().max()
).select(pl.col("step").max()).collect()

step
i64
1


In [22]:
df: pl.LazyFrame = pl.scan_parquet("/home/egor/Projects/Pycharm/kaggle-detect_sleep_states/data/raw/test_series.parquet", low_memory=True)
df

<LazyFrame [5 cols, {"series_id": Utf8 … "enmo": Float32}] at 0x7F95D7299C60>

In [24]:
df.group_by("series_id").agg(pl.col("step").max()).collect()

series_id,step
str,u32
"""0402a003dae9""",149
"""03d92c9f6f8a""",149
"""038441c925bb""",149


In [27]:
df.filter(pl.col("series_id")=="0402a003dae9").collect()

series_id,step,timestamp,anglez,enmo
str,u32,str,f32,f32
"""0402a003dae9""",0,"""2018-12-18T12:…",-77.314903,0.0868
"""0402a003dae9""",1,"""2018-12-18T12:…",-52.1838,0.0434
"""0402a003dae9""",2,"""2018-12-18T12:…",-53.218601,0.0541
"""0402a003dae9""",3,"""2018-12-18T12:…",-51.541698,0.0612
"""0402a003dae9""",4,"""2018-12-18T12:…",-24.6373,0.0591
"""0402a003dae9""",5,"""2018-12-18T12:…",-23.6423,0.0483
"""0402a003dae9""",6,"""2018-12-18T12:…",-18.802099,0.0744
"""0402a003dae9""",7,"""2018-12-18T12:…",-16.5942,0.0657
"""0402a003dae9""",8,"""2018-12-18T12:…",-22.122499,0.0604


In [40]:
df_sub = pd.read_csv("/home/egor/Projects/Pycharm/kaggle-detect_sleep_states/data/submission.csv")
df_sub

,row_id,series_id,step,event,score
0,0,038441c925bb,1,wakeup,0.016281
1,1,038441c925bb,3,onset,0.012810
2,2,038441c925bb,63,onset,0.008789
3,3,038441c925bb,64,wakeup,0.008881
4,4,038441c925bb,105,onset,0.008286
...,...,...,...,...,...
1000,1000,0402a003dae9,8550,onset,0.012367
1001,1001,0402a003dae9,8590,wakeup,0.011101
1002,1002,0402a003dae9,8595,onset,0.010902
1003,1003,0402a003dae9,8637,onset,0.012970


In [44]:
def extract_largest(df: pd.DataFrame) -> pd.Series:
    return df.loc[df["score"].idxmax()]

sub_df = df_sub.groupby([df_sub["series_id"], df_sub["step"] // 360], as_index=False).apply(extract_largest)

sub_df

,row_id,series_id,step,event,score
0,15,038441c925bb,334,wakeup,0.076660
1,22,038441c925bb,501,onset,0.156982
2,38,038441c925bb,892,wakeup,0.128662
3,48,038441c925bb,1105,wakeup,0.059479
4,70,038441c925bb,1671,onset,0.028702
...,...,...,...,...,...
67,940,0402a003dae9,6985,wakeup,0.015541
68,948,0402a003dae9,7206,onset,0.015305
69,971,0402a003dae9,7831,wakeup,0.014503
70,987,0402a003dae9,8241,onset,0.014618


In [45]:
pl.read_parquet("/home/egor/Projects/Pycharm/kaggle-detect_sleep_states/data/raw/test_series.parquet").group_by("series_id").count()

series_id,count
str,u32
"""ba8083a2c3b8""",423900
"""18b61dd5aae8""",495540
"""d5be621fd9aa""",386100
